In [1]:
import requests as req

from requests_oauthlib import OAuth2Session

from oauthlib.oauth2 import BackendApplicationClient
client_id = 'KPh0r2NApri2TY-elJu_DA'
client_secret = '7gy8jEHAZFhif90teGhMNZYkAylFVQtLLE5Qclu544NhALrJNGlTJOJZYHWx5tW1'

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='https://api.yelp.com/oauth2/token', client_id=client_id,
        client_secret=client_secret)
		
print(token['access_token'])

tok = token['access_token']

url_params = {
              'term': 'deli', 
              'location': 'Mission District, San Francisco',
              'limit': 50
             }
url = "https://api.yelp.com/v3/businesses/search"
headers = {
    'Authorization': 'Bearer %s' % tok,
}
busn = req.get(url, headers=headers, params=url_params)

import json
# print(busn.json())
dels = busn.json()

import numpy as np
#First three API calls.
#1 Get API data on review scores. Pandas/Numpy on average review score by restaurant type.
#2 API Call on Reviews that would get average review score and review count by type in San Francisco. 
#3 API Call to get Reviews. Pandas/Dataframe that has sum of reviews by the restaurant type. 

for x in dels['businesses']:
    print(x)
    break

-67Ra6AJMcZ-kNbg6_YwLylMSYqadBS-Sxptx4wlylM9-KFGNVbf0mboOX-QcwD3TcLiq2Bf1h4weBtgy0A2oE_xtmmytRRD2sqKcq2SEzubM8Ns3qATQBjLukKTWXYx
{'id': 'guerrero-market-and-deli-san-francisco', 'name': 'Guerrero Market & Deli', 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/26zh9s2B5brVGf-zPoQOhQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/guerrero-market-and-deli-san-francisco?adjust_creative=KPh0r2NApri2TY-elJu_DA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=KPh0r2NApri2TY-elJu_DA', 'review_count': 91, 'categories': [{'alias': 'grocery', 'title': 'Grocery'}, {'alias': 'delis', 'title': 'Delis'}, {'alias': 'sandwiches', 'title': 'Sandwiches'}], 'rating': 4.5, 'coordinates': {'latitude': 37.7598495483398, 'longitude': -122.423332214355}, 'transactions': [], 'price': '$', 'location': {'address1': '701 Guerrero St', 'address2': '', 'address3': '', 'city': 'San Francisco', 'zip_code': '94110', 'country': 'US', 'state': 'CA', 'display_address': ['701 Guerre

In [12]:
Districts = ["Castro District", "Chinatown", "Ternderloin", "Inner Richmond", "Inner Sunset", "Alamo Square", "Russian Hill", "Mission District", "NorthBeach/Telegraph", "SoMa"]
types = ["Chinese", "Mexican", "French", "Japanese", "Mediterranean", "American", "Italian", "Korean", "Thai", "Indian"]
countreviewsbytype = {}
avgreviewsbytype = {}
totalreviewsbytype = {}

for x in Districts:
    for y in types:
        
        url_params = {
            'term': y,
            'location': x,
            'limit': 50
        }
        
        headers = {
            'Authorization': 'Bearer %s' % tok
        }
        
        reviewcall = req.get(url, headers=headers, params=url_params)
        try:
            reviews = reviewcall.json()
        except:
            print(reviewcall.json())
        
        for rev in reviews['businesses']:
            try:
                if y not in avgreviewsbytype:
                    avgreviewsbytype[y] = rev['rating']
                    countreviewsbytype[y] = 1
                    totalreviewsbytype[y] = rev['review_count']
                else:
                    avgreviewsbytype[y] += rev['rating']
                    countreviewsbytype[y] += 1
                    totalreviewsbytype[y] += rev['review_count']
            except:
                print(rev)

print(countreviewsbytype)
print(avgreviewsbytype)
print(totalreviewsbytype)

{'Chinese': 492, 'Mexican': 500, 'French': 429, 'Japanese': 500, 'Mediterranean': 389, 'American': 500, 'Italian': 500, 'Korean': 488, 'Thai': 487, 'Indian': 392}
{'Chinese': 1797.5, 'Mexican': 1893.0, 'French': 1722.0, 'Japanese': 1961.0, 'Mediterranean': 1520.5, 'American': 2023.0, 'Italian': 2004.5, 'Korean': 1841.0, 'Thai': 1794.5, 'Indian': 1475.5}
{'Chinese': 244488, 'Mexican': 286299, 'French': 354230, 'Japanese': 237149, 'Mediterranean': 167133, 'American': 369758, 'Italian': 358596, 'Korean': 234970, 'Thai': 252265, 'Indian': 176663}


In [27]:
avgscores =  []
comboscores = []
reviewtotal = []

for x in countreviewsbytype:
    count = countreviewsbytype[x]
    avg = avgreviewsbytype[x]
    total = totalreviewsbytype[x]
    avgscores.append(round((avg/count),2))
    comboscores.append(round(((avg/count)*total),2))
    reviewtotal.append(total)
print(avgscores)
print(comboscores)
print(reviewtotal)

[3.65, 3.79, 4.01, 3.92, 3.91, 4.05, 4.01, 3.77, 3.68, 3.76]
[893225.98, 1083928.01, 1421874.27, 930098.38, 653279.5, 1496040.87, 1437611.36, 886433.95, 929547.32, 664964.94]
[244488, 286299, 354230, 237149, 167133, 369758, 358596, 234970, 252265, 176663]


In [2]:
import pandas as pd

In [29]:
avgscoreframe = pd.DataFrame({
    'Food Types': types,
    'Average Scores': avgscores,
})

In [33]:
exportframe = avgscoreframe[['Food Types', 'Average Scores']]

In [34]:
exportframe.to_csv("Yelp/avgscores.csv")

In [35]:
comboscoreframe = pd.DataFrame({
    'Food Types': types,
    'Combo Scores': comboscores,
})

In [36]:
exportframe = comboscoreframe[['Food Types', 'Combo Scores']]
exportframe.to_csv("Yelp/comboscores.csv")

In [37]:
reviewtotalsframe = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': reviewtotal,
})

In [38]:
exportframe = reviewtotalsframe[['Food Types', 'Total Reviews']]
exportframe.to_csv("Yelp/totalreviews.csv")

In [3]:
#Need to get total sum of reviews by neighborhood
#Need to create a dataframe for each neighborhood that captures the review sum by type

In [4]:
Districts = ["Castro District", "Chinatown", "Ternderloin", "Inner Richmond", "Inner Sunset", "Alamo Square", "Russian Hill", "Mission District", "NorthBeach/Telegraph", "SoMa"]
types = ["Chinese", "Mexican", "French", "Japanese", "Mediterranean", "American", "Italian", "Korean", "Thai", "Indian"]
neighbrevs = {}

for x in Districts:
    sumreviews = []
    for y in types:
        
        url_params = {
            'term': y,
            'location': x,
            'limit': 50
        }
        
        headers = {
            'Authorization': 'Bearer %s' % tok
        }
        
        reviewcall = req.get(url, headers=headers, params=url_params)
        try:
            reviews = reviewcall.json()
        except:
            print(reviewcall.json())
        
        revcount = 0
        
        for rev in reviews['businesses']:
            revcount += rev['review_count']
        
        sumreviews.append(revcount)
    
    neighbrevs[x] = sumreviews

print(neighbrevs)

{'Castro District': [18582, 34503, 34639, 16590, 18276, 32310, 33737, 20358, 20786, 20450], 'Chinatown': [29278, 18554, 20212, 14841, 16339, 18599, 23177, 17226, 18811, 11840], 'Ternderloin': [29669, 22165, 48568, 25849, 21692, 40787, 42596, 25858, 26644, 24724], 'Inner Richmond': [20820, 27635, 55636, 23806, 24170, 32864, 32672, 27413, 32076, 25616], 'Inner Sunset': [22587, 31188, 12027, 27423, 6677, 38136, 23030, 33317, 30880, 5131], 'Alamo Square': [16724, 39214, 25367, 22936, 3577, 43004, 36433, 23649, 14459, 5024], 'Russian Hill': [26879, 24818, 36751, 28016, 19487, 40003, 46842, 23642, 28297, 24529], 'Mission District': [19585, 38955, 55325, 19867, 26614, 40350, 36031, 17727, 25290, 27840], 'NorthBeach/Telegraph': [29592, 28352, 12820, 27368, 8283, 46760, 45010, 24142, 27475, 4816], 'SoMa': [30437, 19968, 52402, 30596, 22189, 34229, 39708, 22092, 27691, 26653]}


In [7]:
Castro = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': neighbrevs['Castro District']
})

Chinatown = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': neighbrevs['Chinatown']
})

Tenderloin = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': neighbrevs['Ternderloin']
})

Inner_Richmond = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': neighbrevs['Inner Richmond']
})

Inner_Sunset = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': neighbrevs['Inner Sunset']
})

Alamo_Square = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': neighbrevs['Alamo Square']
})

Russian_Hill = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': neighbrevs['Russian Hill']
})

Mission_District = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': neighbrevs['Mission District']
})

NorthBeach = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': neighbrevs['NorthBeach/Telegraph']
})

SoMa = pd.DataFrame({
    'Food Types': types,
    'Total Reviews': neighbrevs['SoMa']
})

In [10]:
Castro.to_csv("Yelp/Castro.csv")
Chinatown.to_csv("Yelp/Chinatown.csv")
Tenderloin.to_csv("Yelp/Tenderloin.csv")
Inner_Richmond.to_csv("Yelp/InnerRichmond.csv")
Inner_Sunset.to_csv("Yelp/InnerSunset.csv")
Alamo_Square.to_csv("Yelp/AlamoSquare.csv")
Russian_Hill.to_csv("Yelp/RussianHill.csv")
Mission_District.to_csv("Yelp/MissionDistrict.csv")
NorthBeach.to_csv("Yelp/NorthBeach.csv")
SoMa.to_csv("Yelp/Soma.csv")